In [ ]:
from datetime import datetime, timedelta

import numpy as np
from skyfield import almanac
from skyfield.api import Angle, N, W, Loader, PlanetaryConstants, utc, wgs84
from skyfield.framelib import ecliptic_frame
from skyfield.trigonometry import position_angle_of

In [ ]:
latitude = (35 + 58/60 + 10/3600) * N
longitude = (84 + 19/60) * W
# obs_date_time = datetime(2013, 10, 18, 22, 0, 0)
# obs_date_time = obs_date_time.replace(tzinfo=utc)
# obs_midnight = obs_date_time.date()
# obs_midnight_next = obs_midnight + timedelta(days=1)

In [ ]:
load = Loader("~/skyfield")
ts = load.timescale(builtin=False)

In [ ]:
eph = load("de421.bsp")
moon, sun, earth = eph["Moon"], eph["Sun"], eph["Earth"]

In [ ]:
pc = PlanetaryConstants()
pc.read_text(load('moon_080317.tf'))
pc.read_text(load('pck00008.tpc'))
pc.read_binary(load('moon_pa_de421_1900-2050.bpc'))
moon_frame = pc.build_frame_named('MOON_ME_DE421')

In [ ]:
location = wgs84.latlon(latitude, longitude)
topos = earth + location

In [ ]:
obs_date = ts.utc(2013, 11, 10, 22, 0, 0)
tmp = obs_date.utc_datetime()
max_minutes = 1440 * 1
obs_times = ts.utc(tmp.year, tmp.month, tmp.day, 0, range(0, max_minutes, 10), 0)

In [ ]:
astrometric = topos.at(obs_times).observe(moon)
apparent = astrometric.apparent()
ra, dec, distance = apparent.radec('date')
alt, az, _ = apparent.altaz()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(obs_times.utc_datetime(), alt.degrees, 'o')

In [ ]:
plt.plot(obs_times.utc_datetime(), np.sin(dec.radians), 'o')

In [ ]:
plt.plot(obs_times.utc_datetime(), np.cos(dec.radians), 'o')

In [ ]:
plt.plot(obs_times.utc_datetime(), ra.hours, 'o')

In [ ]:
sin_lat = np.sin(np.deg2rad(latitude))
cos_lat = np.cos(np.deg2rad(latitude))
print(sin_lat, cos_lat)

In [ ]:
def altitude_fit(t, a, b, c, d):
    return a * c + b * d * np.cos(t)

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
times = np.array([d.timestamp() for d in obs_times.utc_datetime()])
times = times - times[0]
times *= (2 * np.pi) / (24 * 3600)
# print(times)
fixed_alt = lambda t, c, d: altitude_fit(t, sin_lat, cos_lat, c, d)
# bounds = ((sin_lat, -np.inf, cos_lat, -np.inf), (sin_lat+0.001, np.inf, cos_lat+0.001, np.inf))
param, param_cov = curve_fit(fixed_alt, times, np.sin(alt.radians))

In [ ]:
param

In [ ]:
alt_fit = altitude_fit(times, sin_lat, cos_lat, *param)

In [ ]:
plt.plot(obs_times.utc_datetime(), np.rad2deg(np.arcsin(alt_fit)))

In [ ]:
hour_angle = obs_times.gmst + longitude/15 - ra.hours

In [ ]:
hour_angle *= 15

In [ ]:
alt_func = np.arcsin(sin_lat * np.sin(dec.radians) + cos_lat * np.cos(dec.radians) * np.cos(np.deg2rad(hour_angle)))

In [ ]:
plt.plot(obs_times.utc_datetime(), np.rad2deg(alt_func))

In [ ]:
diff = alt.degrees - np.rad2deg(alt_func)

In [ ]:
plt.plot(times, diff)

In [ ]:
def alt_fit2(t, a, b, c, d, e, f, g, h, i, j):
    return a * np.sin(e * t + f) * np.sin(g * t + i) + np.cos(b * t + c) * np.cos(h * t + j) + d

In [ ]:
def alt_fit3(t, a, b, c, d, e, f, g):
    return a * np.sin(c * t + d) + b * np.cos(e * t + f) * np.cos(g * t)

In [ ]:
fixed_alt3 = lambda t, c, d, e, f, g: alt_fit3(t, sin_lat, cos_lat, c, d, e, f, g)

In [ ]:
param, param_cov = curve_fit(alt_fit2, times, alt.degrees,
                             p0=(np.max(alt.degrees), 1/1440, 1, 1/14400, 1, 1, 0.1, 0.1, 0.1, 0.1),
                             maxfev=1000000)

In [ ]:
param

In [ ]:
plt.plot(times, alt.degrees, 'o')
plt.plot(times, alt_fit2(times, *param))